In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Ollama for subquery generation on training set
- same code is extended for subquery generation on the validation and test set

In [ ]:
#Download ollama
!curl -fsSL https://ollama.com/install.sh | sh
import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread
#Download model
!ollama pull llama3
!pip install ollama

In [ ]:
import json
import ollama

# Define the function to decompose claims into subqueries using Ollama
def decompose_claim(claim):
    messages = [
        {
            'role': 'user',
            'content': f"Decompose the following claim into subquestions and return only the concatenated subqueries with nothing else in the response:\n\n{claim}\n\n",
        },
    ]
    response = ollama.chat(model='llama3', messages=messages)
    
    return response['message']['content']

# Load the JSON data
input_file_path = '/kaggle/input/quantemp-training/train_claims_quantemp.json'
output_file_path = '/kaggle/working/ollama-decomposed-train.json'

with open(input_file_path, 'r') as file:
    data = json.load(file)

processed_data = data
# Iterate over each entry, decompose the claim and add subqueries
iteration_count = 0
# Iterate over each entry, decompose the claim and add subqueries
for idx, entry in enumerate(processed_data, start=0):
    claim = entry['claim']
    subqueries = decompose_claim(claim)
    entry['subqueries'] = subqueries

    # Print subqueries after every 500 iterations
    if (idx + 1) % 500 == 0:
        iteration_count += 1
        print(f"Iteration {iteration_count * 500}: Subqueries added: ",subqueries)

# Save the updated data back to a new JSON file
with open(output_file_path, 'w') as file:
    json.dump(processed_data, file, indent=4)

print("Subqueries added and JSON file updated successfully.")
